%pip install transformers==4.38.2
%pip install peft==0.10.0
%pip install sentencepiece==0.2.0
%pip install accelerate==0.28.0
%pip install -i https://pypi.org/simple/ bitsandbytes

%pip install --force-reinstall chromadb==0.4.23 
%pip install --force-reinstall llama_index==0.10.12
%pip install --force-reinstall sentence_transformers==2.2.2

In [25]:
from typing import List
import json
import chromadb
from llama_index.core import ServiceContext, StorageContext
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from llama_index.llms.openai import OpenAI
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, GenerationConfig
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import ast
import torch
from tqdm.auto import tqdm
set_seed(42)

In [26]:
MODEL_NAME = 'IlyaGusev/saiga_mistral_7b'

#MODEL_NAME ="openchat/openchat-3.5-0106"

EMBED_MODEL = 'intfloat/e5-large-v2'
EMBED_MODEL = 'BAAI/bge-small-en-v1.5'

In [27]:
import torch

# Get the properties of the CUDA device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
properties = torch.cuda.get_device_properties(device)

# Print the total memory and memory available on the CUDA device
total_memory = properties.total_memory / 1024**2  # Convert to megabytes
memory_available = torch.cuda.memory_reserved(device) / 1024**2  # Convert to megabytes

print(f"Total video memory: {total_memory:.2f} MB")
print(f"Available video memory: {memory_available:.2f} MB")

Total video memory: 7951.19 MB
Available video memory: 576.00 MB


Иницилизируем модель для получения эмббедингов

In [28]:
embed_model = SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL, 
                                                device='cuda'
                                                )

Создаем коллекцию в векторном хранилище

In [29]:
client = chromadb.Client()
collection = client.get_or_create_collection("RAG", embedding_function=embed_model,
    metadata={"hnsw:space": "cosine"} 
    )
client.list_collections()

[Collection(name=RAG)]

Загружаем данные

In [30]:
#file_path = "triples_ft_pp.jsonl"
file_path = "data/movie_ground_truth.jsonl" # больше данных - больше вариативность

with open(file_path, "r") as f:
    data = f.readlines()

In [31]:
documents = []
for i_text in data:
    i_text = json.loads(i_text)
    documents.append(i_text['sent'])

In [32]:
metadata = [{'subject': 'movie'} for i in range(len(documents))]

### Грузим каталог

In [33]:
df = pd.read_csv('../lesson12/data/data.csv_')
df.head()

,title,url,price,text
0,Возрожденное будущее Т Хига,https://em-russia.ru/shop/all/vozrozhdennoe-bu...,0,NaN
1,Брошюра Природное земледелие с ЭМ,https://em-russia.ru/shop/all/broshyura-prirod...,191,NaN
2,ЭМ БИО концентрат 1 ВОСТОК ЭМ 1,https://em-russia.ru/shop/all/em-bio-kontsentr...,780,Набор для самостоятельного активирования (приг...
3,ЭМ 5 биорегулятор болезни вредители,https://em-russia.ru/shop/all/em-5-bioregulyat...,515,Предназначен для предотвращения заболеваний и ...
4,ОФЭМ удобрение ЭМ Бокаси,https://em-russia.ru/shop/all/ofem-udobrenie-e...,565,Универсальное органическое удобрение производи...


In [41]:
documents = []
metadata = []
for index, row in df.iterrows():
    
    documents.append(row['title'])
    metadata.append({'url' : row['url']})
    
    documents.append(row['text'])
    metadata.append({'url' : row['url']})

### Грузим статьи

In [42]:
import os
for filename in os.listdir('../lesson12/data/page_texts/'):
    with open('../lesson12/data/page_texts/' + filename, 'r') as f:
        sentenses = f.read()
        
        #for sentense in sentenses:
        documents.append(sentenses)
        metadata.append({'url' : f'https://em-russia.ru/base/{filename.split(".")[0]}/'})
        
    

In [43]:
metadata

Подготоавливаем данные для загрузки в векторную бд

In [12]:
metadata = [{'type': 'product'} for i in range(len(documents))]

In [13]:
ids = ['id'+str(collection.count()+i+1) for i in range(len(documents))]

In [14]:
collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids,
)


In [15]:
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
torch_device = "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, 
                                             pad_token_id=tokenizer.eos_token_id).to(torch_device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# model.generation_config = GenerationConfig(
#        do_sample=True,
#        top_k=50,
#        top_p=0.95,
#        num_return_sequences=num_return_sequences, 
#        num_beams=num_beams,
#        temperature=0.5
#    )

Промпт для RAG

In [17]:
prompt = """Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query}
Answer:"""

Функция для поиска контекста

In [18]:
def get_context(query):
    context_raw = collection.query(query_texts=query, n_results=3)
    context = "\n".join(context_raw['documents'][0])
    return context

Функция для генерации нескольких ответов модели

Закомментированные параметры - эксперимент

In [19]:
def get_model_output(context, query):
    model_inputs = tokenizer(prompt.format(context=context, query=query), return_tensors='pt').to(torch_device)

    sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=50,
    do_sample=False, # True,
    #top_k=50,
    #top_p=0.95,
    num_return_sequences=3, #6, 
    num_beams=3, #6,
    temperature=None
    )

    result = []
    for i, output in enumerate(sample_outputs):
        result.append(tokenizer.decode(output, skip_special_tokens=True).split("Answer:")[1].strip().split("\n")[0].strip())
    return result

In [20]:
def get_output_texts(query: str) -> List[str]:
    context = get_context(query)
    output_texts = get_model_output(context, query)
    return output_texts

Тест

In [21]:
get_output_texts("Чем лучше убирать помещения?")

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


['Убирать помещения можно разными способами, но наиболее эффективным и безопасным способом является использование вакуумной уборки. Ваку',
 'Убирать помещения можно разными способами, но наиболее эффективным и безопасным способом является использование вакуум-чистки. Ваку',
 'Убирать помещения можно разными способами, но наиболее эффективным и безопасным способом является использование вакуум-чистки. Это']

In [22]:
#df = pd.read_csv('eval_dataset.csv')

In [23]:
#df = df.head(3)
#df

Получение выходов<br>
Тут сделан алгоритм, который запускает модель несколько раз, чтобы выбрать ответ с отличиющимися ответами<br>
Но когда температура выключена, то повтор не имеет смысле (закомментирован)<br>
Экспериментально, отключение семплинга позволяет beam search выдавать более разнообразные ответы<br>
Поэтому закомментирвоано как эксперимент

In [24]:
def f7(seq):
    """Remove duplicates from a list, while preserving order """
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

num_return_sequences = 3

data = []
for query in tqdm(df.question):
    # когда нет семплинга
    data.append(get_output_texts(query)[:num_return_sequences]) 
    
    # когда есть семплинг
    # max_unique = 0
    # best_outputs = [''] * num_return_sequences
    # for _ in range(5):
    #     outputs = get_output_texts(query)
    #     unique_count = len(set(outputs))
    #     if unique_count > max_unique:
    #         max_unique = unique_count
    #         best_outputs = outputs
    #         
    #     if unique_count >= num_return_sequences:
    #         break
    #         
    #     if unique_count == 1 and max_unique == 1:
    #         break
    #         
    # best_outputs = f7(best_outputs)[:num_return_sequences]
    # if len(best_outputs) != num_return_sequences:
    #     best_outputs += [best_outputs[-1]] * (num_return_sequences - len(best_outputs))
    #        
    #data.append(best_outputs)

AttributeError: 'DataFrame' object has no attribute 'question'

Сохранение

In [ ]:
columns = [f'rag_{n}' for n in range(num_return_sequences)]
data_t = list(map(list, zip(*data)))

for i, col in enumerate(columns):
    df[col] = data_t[i]

IndexError: list index out of range

In [ ]:
def uniq_count(r):
    return len(set([r['rag_0'], r['rag_1'], r['rag_2']]))

df['uniq_count'] = df.apply(uniq_count, axis=1)
display(df['uniq_count'].value_counts())

df.to_csv('eval_dataset.csv', index=False)

uniq_count
2    63
3    51
1    43
Name: count, dtype: int64